In [64]:
import requests
from lxml import etree
from selenium import webdriver
import re
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [81]:
class LagouSpider(object):
    driver_path = r'L:\download\chromedriver.exe'
    def __init__(self):
        self.driver = webdriver.Chrome(executable_path=LagouSpider.driver_path)
        self.url = 'https://www.lagou.com/jobs/list_python?labelWords=&fromSearch=true&suginput='
        self.positions = []
        
    def run(self):
        self.driver.get(self.url)
        while True:
            source = self.driver.page_source
#             WebDriverWait(driver=self.driver_path, timeout=10).until(
#                 EC.presence_of_element_located((By.XPATH,"//div[@class='pager_container']//span[last()]"))
#             )
            self.parse_list_page(source)
            time.sleep(0.5)
            try:
                btn = self.driver.find_element_by_xpath("//div[@class='body-box']//div[@class='body-btn']")
                btn.click()
            except:
                pass
            next_btn = self.driver.find_element_by_xpath("//div[@class='pager_container']//span[last()]")
            if 'pager_next pager_next_disabled' in next_btn.get_attribute('class'):
                break
            else:
                next_btn.click()
                time.sleep(1)
        
    def parse_list_page(self,source):
        html = etree.HTML(source)
        links = html.xpath("//a[@class='position_link']/@href")
        for link in links:
            self.request_detail_page(link)
            time.sleep(1)
        
    def request_detail_page(self,url):
        #self.driver.get(url)
        self.driver.execute_script("window.open('%s')" % url)
        self.driver.switch_to.window(self.driver.window_handles[1])
        source = self.driver.page_source
        self.parse_detail_page(source)
        self.driver.close()
        self.driver.switch_to.window(self.driver.window_handles[0])        
        
        
    def parse_detail_page(self,source):
        html = etree.HTML(source)
        position_name = html.xpath("//h1[@class='name']/text()")[0]
        job_request_spans = html.xpath("//dd[@class='job_request']//span")
        salary = job_request_spans[0].xpath(".//text()")[0].strip()
        city = job_request_spans[1].xpath(".//text()")[0]
        city = re.sub(r"[\s/]", "", city)
        work_years = job_request_spans[2].xpath(".//text()")[0]
        work_years = re.sub(r"[\s/]", '', work_years)
        education = job_request_spans[3].xpath(".//text()")[0]
        education = re.sub(r"[\s/]", '', education)
        type_ = job_request_spans[4].xpath(".//text()")[0]
        describe = ''.join(html.xpath("//div[@class='job-detail']//text()")).strip()
        position = {
            'name':position_name,
            'salary':salary,
            'city':city,
            'work_years':work_years,
            'education':education,
            'describe':describe
        }
        self.positions.append(position)
        print(position)
        print("="*30)
    
spider = LagouSpider()
spider.run()